# Expression Evaluation
 
***Student Name:*** put your name here
    
    
## Submission

After answering all the questions, save your work in **PDF** format file (if saving in PDF format is not working submit Notebook-formated `.ipynb` version).
    
- Double-click on this cell
- Enter your name in the above placeholder, and evaluate this cell to render it correctly
- Save your work by pressing <span class="fa-save fa"/> button in the toolbar
- Go to menu "File" -> "Download as"
- Select "PDF via Latex (.pdf)"
- Use downloaded file for Blackboard submission 

For more information, see https://www.codecademy.com/articles/how-to-use-jupyter-notebooks

### Coding Style

- Use functional F# style for writing your programs.
- Make sure that you do not use mutable variables & loops.
- Any imperative style programming is prohibited unless specified in the problem description.

For additional information of F# coding style see [F# Style Guide](https://docs.microsoft.com/en-us/dotnet/fsharp/style-guide/).

### Before You Submit

You are required to test that your submission works properly before submission. Make sure that your program compiles without errors. Once you have verified that the submission is correct, you can submit your work.

### Your Submission

Program submissions should be done through the Blackboard.
    

## Problems

In this assignment, you will write a lexical analyzer and an interpreter for string expressions.



Write a lexical analyzer which uses above types to generate sequence of tokens from the input string. You will tokenize strings composed of integer numbers and arithmetic operations. Let's define these tokens.

In [ ]:
//
// RUN THIS CELL BEFORE CONTINUE
//
type TOKEN =
    | INT of int // integer number
    | ADD_OP   // addition operation
    | SUB_OP   // subtraction operation
    | MUL_OP   // multiplication operation
    | UNKNOWN  // uknown symbol
    | EOF      // end of input
    
// helper functions
let charsToInt (cs:char list) :int =
    List.fold (fun a c -> a + string c) "" cs |> int

The lexical analyzer is a finite automaton (finite state machine) that takes a string and produces a sequence of tokens corresponding to proper language elements in the string.

![](lexer.png)

The transition between the states can be defined as a collection of recursive functions. Each function represents a series of transitions that happen in the state of a finite automaton. For example:

- In `START` state, it's possible to go to `OP` state if an arithmetic operation character is received, or go to `NUMBER` state if the input starts with a number
- In `OP` state, it's *only* possible to go to `SPACE` state if the input starts from the space character.


When finite automaton leaves a particular state, the appropriate token is generated. That corresponds to the same action in the recursive function. Leaving the state is associated with a function termination, and so a function returns a *list of tokens* which where accumulated through the recursive calls attaching the latest generated token to the beginning of the list.

If finite automaton receives a character that does not correspond to the appropriate transition then the function must return the `UNKNOWN` token.

### Lexer (20 pts)

**RUN ABOVE CELLS BEFORE CONTINUE**

First, write some helper function that will help to write the lexical analyzer. A function that identifies if a character is a number.

In [ ]:
// This function classifies whether a character is a number or not.
let isNumber (c:char) :bool =
    ...

And, a function that identifies if a character is an arithmetic operation.

In [ ]:
// This function classifies whether a character is an arithmetic operation or not.
let isOperation (c:char) :bool =
    ...

There are already `start` and `number` functions defined as an examples of the state transitions. Write the rest of the functions in a similar manner to complete the automaton program.

*Note: Because all functions are recursive and they call each other, the F# provides special syntax for such [mutually recursive function](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/functions/recursive-functions-the-rec-keyword#mutually-recursive-functions).*

In [ ]:
let rec start (input:char list) =    
    match input with
    | c::str when isNumber c    -> (number [c] str)
//    | c::str when isOperation c -> (operation c str)
    | []                        -> [EOF]
    | _                         -> [UNKNOWN]
    
and number (nums:char list) (input:char list) =
    let makeIntLit cs = cs |> List.rev |> charsToInt |> INT
    match input with
    | []       -> (makeIntLit nums)::[EOF]      
//    | ' '::str -> (makeIntLit nums)::(space str)    
    | c::str when isNumber c -> (number (c::nums) str)
    | _        -> [UNKNOWN]
       
//and space (input:char list) =
//    ...
    
//and operation (c:char) input =
//    ...

The `lexer` function take an input string with calls the above finite automaton functions. The result of this function is a list of tokens.

In [ ]:
let lexer (input:string) :TOKEN list =
    input |> Seq.toList |> start 

Here is some tests

In [ ]:
lexer "123" |> printfn "%A" // [INT 123; EOF]

In [ ]:
lexer "10 + 20 - 2 * 10" |> printfn "%A" // [INT 10; ADD_OP; INT 20; SUB_OP; INT 2; MUL_OP; INT 10; EOF]

In [ ]:
lexer "10 + 20 - a" |> printfn "%A" // [INT 10; ADD_OP; INT 20; SUB_OP; UNKNOWN]

### Expression Evaluation (10 pts)

There are three different ways how the expressions can be represented:

- **Prefix expression** has the form `<op> <a> <b>` where an operator `op` is preceded the every pair of operands `a` and `b`.
- **Infix expression**  has the form `<a> <op> <b>` where an operator `op` is in-between every pair of operands `a` and `b`.
- **Postfix expression** has the form `<a> <b> <op>` where an operator `op` is followed for every pair of operands `a` and `b`.


You will write an `infix` function that evaluates sequence of tokens in infix form using the input from the lexical analyzer, and output integer result value. All binary operations have the same precedence, and they are evaluated from left to right.

In [ ]:
let rec infix (tkns:TOKEN list) :int =
    ...

Test your function

In [ ]:
"10 * 20 + 30 - 40" |> lexer |> infix // 190

### Bonus (15 pts)

Write a `postfix` function which evaluates sequence of tokens in postfix notation form using the input from the lexical analyzer, and output integer result value. 

In [ ]:
let postfix (tkns:TOKEN list) :int =
    ...

Test your function

In [ ]:
"10 5 20 10 + - *" |> lexer |> postfix // -250

In [ ]:
"10 20 + 5 - 10 *" |> lexer |> postfix // -250

In [ ]:
"1 3 + 5 2 - *" |> lexer |> postfix // 12